In [1]:
import numpy as np
import numpy_linalg as la
import complex_synapse as cs
from sl_py_tools.import_tricks import reload

In [2]:
np.set_printoptions(formatter={'float_kind': '{: #5.2g}'.format})

In [101]:
%pdoc cs.SynapseMemoryModel

In [10]:
reload(cs)

Reloading complex_synapse.synapse_memory_model
Reloading complex_synapse.synapse_base
Reloading complex_synapse.builders
Reloading sl_py_tools.numpy_tricks
Reloading complex_synapse.synapse_opt


In [11]:
def balance_grad(model: cs.SynapseMemoryModel, s: float) -> float:
    W = model.markov()
    Zi = model.zinv(s)
    K = model.enc()
    p = model.peq()
    Zid = cs.builders.ma.adjoint(Zi, p)
    return (p.r @ K @ Zi.inv @ (la.eye(model.nstates) - Zid @ Zi.inv) @ model.weight.c).us

In [4]:
s = 4
n = 6

In [14]:
syn = cs.SynapseMemoryModel.rand(n, binary=True)
syn.sign_fix()
ss = la.linspace(0, s, 10)
bgrad = balance_grad(syn, ss)
# snrl = syn.snr_laplace(ss)
print(f'gradient {bgrad * 100}/100')
# print(f'laplace  {snrl * 100}/100')
print('sign', (bgrad >= 0).all())

gradient [-0.70 -0.61 -0.53 -0.46 -0.41 -0.36 -0.32 -0.28 -0.25 -0.23]/100
sign False


In [133]:
syn.enc()

lnarray([[ 0.52, -0.0067, -0.16, -0.31,  0.079, -0.13],
         [-0.43,  0.55, -0.16, -0.24,  0.30, -0.014],
         [ 0.26,  0.057, -0.49,  0.39, -0.33,  0.12],
         [-0.31,  0.014, -0.36,  0.24,  0.28,  0.15],
         [-0.19,  0.084,  0.24,  0.45, -0.30, -0.28],
         [-0.011, -0.17, -0.25,  0.36, -0.069,  0.14]])